# Dados sobre instituições religiosas no Brasil

De acordo com o [art. 150, VI da Constituição Federal](http://www.planalto.gov.br/ccivil_03/constituicao/constituicao.htm#art150vi), é vedado à União, aos Estados, ao DF e aos municípios instituir impostos sobre templos de qualquer culto, regra que abrange também rendas e serviços relacionados à sua entidade mantenedora. Trocando em miúdos, isto significa que além de não pagarem impostos sobre aluguel de imóveis, bens em nome da entidade e serviços prestados, entidades religiosas também não sofrem tributação.

Com essa imunidade, as igrejas não pagam IPTU (Imposto Predial e Territorial Urbano), Imposto de Renda sobre o que arrecadam em dízimo, IPVA (Imposto Sobre Propriedade de Veículos Automotores) sobre os veículos que possuem e ISS (Imposto Sobre Serviços). ([fonte](https://economia.uol.com.br/noticias/redacao/2019/11/26/comissao-isenta-igreja-de-icms-por-15-anos-texto-vai-ao-plenario-do-senado.htm?cmpid=copiaecola))

Baseado nos dados abertos pela receita federal no dia 11/Novembro aqui, extrai apenas entidades com CNAE **94.91-0-00** - `Atividades de organizações religiosas ou filosóficas` para analisar estruturalmente a disposição das instituições religiosas no país e encontrar alguns padrões dos mesmos.


In [1]:
# antes de mais nada, vou fazer todos os imports que serão necessarios
# para esta análise, deixando o documento mais limpo e legível
import pandas as pd
import sqlite3
import folium
from folium import plugins
import matplotlib.pyplot as plt
import seaborn as sns
# carregar o banco gerado
conn = sqlite3.connect("./religiao.db")
#
%matplotlib inline

## Arquivos

Não disponibilizarei os arquivos aqui por serem todos grandes, mas descreverei passo a passo como gerar cada um deles.

Para baixar a base principal, usei um simples script:
```
for num in 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20
do
  aria2c -s 16 -x 16 "http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_$num.zip"
done
```
Para processar, pode se usar o excelente repositório [socios-brasil](https://github.com/turicas/socios-brasil) do turicas.

Depois de obter o banco, *socios-brasil.sqlite* , exportei o apenas o CNAE correto para um outro banco.

```
sqlite3 -csv socios-brasil.sqlite "select * from empresa where cnae_fiscal=9491000;" > empresas.csv

sqlite3 -csv socios-brasil.sqlite "select s.* from empresa_socia s, empresa e where s.cnpj = e.cnpj and e.cnae_fiscal=9491000;" > empresas_socias.csv

sqlite3 -csv socios-brasil.sqlite "select s.* from socio s, empresa e where s.cnpj = e.cnpj and e.cnae_fiscal=9491000;" > socios.csv
```
E depois criei um novo sqlite com a inserção destes 3 *csv*s.

Para as cidades, usei os dados disponibilizados pelo IBGE em [seu ftp](ftp://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/localidades).

Depois tive de tratar o nome dos municipios para coincidir com os nomes no arquivo da receita, maiusculos e sem acentos, com:
```
>>> import pandas as pd
>>> csv_input = pd.read_csv('municipios.csv')
>>> from unicodedata import normalize
>>> fixname = lambda _txt: normalize('NFKD', _txt).encode('ASCII', 'ignore').decode('ASCII').upper()
>>> csv_input['municipio'] = csv_input['nome'].map(fixname)
>>> csv_input.to_csv('mod.csv', index=False)
```
o resultado foi:

In [2]:
igjs = pd.read_csv('./mod.csv')

In [3]:
igjs.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,municipio
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,ABADIA DE GOIAS
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,ABADIA DOS DOURADOS
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,ABADIANIA
3,3100203,Abaeté,-19.15510,-45.4444,0,31,ABAETE
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,ABAETETUBA


finalmente, removi os dados que nao precisava e acabei gerando uma versão dos dados agrupando as instituições por município

In [4]:
bloco = pd.read_csv('./per_latlon.csv')

In [5]:
bloco.head()

,uf,municipio,qtde,latitude,longitude
0,AC,ACRELANDIA,13,-9.82581,-66.8972
1,AC,ASSIS BRASIL,5,-10.92980,-69.5738
2,AC,BRASILEIA,20,-10.99500,-68.7497
3,AC,BUJARI,10,-9.81528,-67.9550
4,AC,CAPIXABA,8,-10.56600,-67.6860


A partir destes arquivos e do banco sqlite3 que gerei, começei a fazer algumas inferencias.

## Alguns dados numéricos

- Numero de entradas como instuição religiosas:
```
sqlite3 religiao.db "select count(*) from igrejas;"
```
**255369**
- Instituições por estado:
```
sqlite3 religiao.db "select uf, count(*) from igrejas group by uf;"
```
```
| UF | qtde |
|----|------|
| AC | 1007 |
| AL | 2266 |
| AM | 3558 |
| AP | 821  |
| BA | 13841|
| CE | 5789 |
| DF | 5329 |
| ES | 7447 |
| EX | 57   | <-- atenção especial a isso, instituições fora do país
| GO | 10222|     porém registradas e recolhendo(no caso não recolhendo)
| MA | 3811 |     no Brasil.
| MG | 26852|
| MS | 4600 |
| MT | 3684 |
| PA | 4959 |
| PB | 3253 |
| PE | 6852 |
| PI | 1725 |
| PR | 15004|
| RJ | 37598|
| RN | 1968 |
| RO | 2184 |
| RR | 498  |
| RS | 11897|
| SC | 6974 |
| SE | 1956 |
| SP | 69333|
| TO | 1884 |
```
- instuições fora do brasil:

In [6]:
df2 = pd.read_sql_query("select razao_social, nome_cidade_exterior, nome_pais from igrejas where uf = 'EX';", conn)
df2

,razao_social,nome_cidade_exterior,nome_pais
0,MINISTERIO JESUS CRISTO E A PORTA - MJCP,COCHABAMBA,BOLIVIA
1,IGREJA EVANGELICA BOLA DE NEVE,LOS ANGELES,ESTADOS UNIDOS
2,IGREJA EVANGELICA BOLA DE NEVE,LONDRES,INGLATERRA
3,IGREJA EVANGELICA BOLA DE NEVE,TORONTO,CANADA
4,IGREJA EVANGELICA BOLA DE NEVE,CONCELHO E DISTRITO DE LISBOA,PORTUGAL
5,IGREJA EVANGELICA BOLA DE NEVE,MATOLA,MOCAMBIQUE
6,IGREJA EVANGELICA BOLA DE NEVE,PORTO PRINCIPE,HAITI
7,IGREJA EVANGELICA BOLA DE NEVE,MILILANI,ESTADOS UNIDOS
8,IGREJA EVANGELICA BOLA DE NEVE,DUBLIN,IRLANDA
9,IGREJA EVANGELICA BOLA DE NEVE,CIUDAD DE PANAMA,PANAMA


## Tabela socio

Uma coisa super interessantes da base da receita federal é uma tabela que mostra o quadro societário das instituições religiosas. Algumas informações interessantes partiram disto, como uma unica pessoa registrando uma enorme quantidade de igrejas com cnpjs distintos.

- Top 30 pessoas que mais registraram instituições religiosas:

In [7]:
df3 = pd.read_sql_query("select s.nome_socio, count(*) as qtd from socio s, igrejas i where i.cnpj = s.cnpj group by s.nome_socio order by qtd desc LIMIT 30;", conn)
df3

,nome_socio,qtd
0,MAURICIO CESAR CAMPOS SILVA,9665
1,MARIO DE OLIVEIRA,5495
2,JOSIAS GONCALVES DA CRUZ,2620
3,LUCIANO RAMOS NOGUEIRA DA SILVEIRA,2619
4,MATEUS MACHADO DE OLIVEIRA,2501
5,DANIEL MOREIRA,2400
6,ANTONIO CARLOS DE CAMARGO,1670
7,ALEXANDRE ROCHA MAIA,776
8,EDSON ERTHAL DE MEDEIROS,729
9,EMMANUEL OLIVEIRA GUIMARAES,724


E entre as instituições fora do brasil, vemos uma concentração similar:

In [8]:
df4 = pd.read_sql_query("select s.nome_socio, count(*) as qtd from socio s, igrejas i where i.cnpj = s.cnpj and uf = 'EX' group by s.nome_socio order by qtd desc LIMIT 30;", conn)
df4

,nome_socio,qtd
0,RINALDO LUIZ DE SEIXAS PEREIRA,21
1,ROY NEMOTO,1
2,R. SCOTT GALLAGHER,1
3,PAT SCORT,1
4,KONGTRUL JIGME NAMGYEL,1
5,JONAS ABIB,1
6,JOAQUIM CANCIO PEREIRA SOARES,1
7,IZAIAS DE SOUSA MACIEL,1
8,DAVID DA COSTA,1
9,ALFREDO BENEDICTO PORRATI LAZO,1


Ha muitas outras análises a se retirar disto, mas vamos partir para algo mais gráfico.

É interessante ver um heatmap de instituições para entender visualmente sua concentracao:

In [9]:
# setar o mapa no "meio" do brasil
m = folium.Map([-14.2350, -51.9253], zoom_start=4)
# uma coisa importante, usei dropna() para remover alguns NaNs gerados por
# não exister a lat/log de algumas cidades no banco do IBGE que, mesmo
# sendo de 2018, tem várias cidades faltando.
igrejas = bloco.dropna()[['latitude','longitude','qtde']].values
m.add_child(plugins.HeatMap(igrejas, radius=15))

## Proximos passos
- extrair dados das regiões e correlacionar com a quantidade de instituições religiosas;
  - abortos;
  - desenvolvimento social;
  - escolaridade;
  - votos;
  - impostos coletados;
  - crimes;
- verificar a situação das instituições no exterior;
- verificar a situação das pessoas que registraram as instituições;